In [139]:
import pandas as pd
import numpy as np
import os

# 1. annotation 파일

In [140]:
df_ann = pd.read_csv('C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/annotation/Sess01_eval.csv')

df_ann.drop(columns=["Numb"], axis=1, inplace=True)

In [141]:
df_ann.head()

,WAV,,Segment ID,Total Evaluation,.1,.2,Eval01F,.3,.4,Eval02M,...,.16,Eval08M,.17,.18,Eval09F,.19,.20,Eval10M,.21,.22
0,start,end,,Emotion,Valence,Arousal,Emotion,Valence,Arousal,Emotion,...,Arousal,Emotion,Valence,Arousal,Emotion,Valence,Arousal,Emotion,Valence,Arousal
1,3.5783901,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9,neutral,3,3,neutral,...,3,neutral,4,3,neutral,3,3,neutral,3,3
2,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9,neutral,3,3,neutral,...,3,neutral,4,3,neutral,3,3,neutral,3,3
3,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3,neutral,3,3,neutral,...,3,neutral,4,3,neutral,3,3,neutral,3,3
4,31.558392,42.97039,Sess01_script01_User002M_004,neutral,3.7,3.1,neutral,4,3,neutral,...,3,neutral,4,4,neutral,3,3,neutral,4,3


### Total Evaluation까지만 남기고 뒤 컬럼들 모두 제거

In [142]:
column_to_start_deleting = 'Eval01F'
start_index = df_ann.columns.get_loc(column_to_start_deleting)

columns_to_drop = df_ann.columns[start_index:].tolist()
df_ann.drop(columns=columns_to_drop, axis=1, inplace=True)

In [143]:
df_ann.head()

,WAV,,Segment ID,Total Evaluation,.1,.2
0,start,end,,Emotion,Valence,Arousal
1,3.5783901,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9
2,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9
3,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3
4,31.558392,42.97039,Sess01_script01_User002M_004,neutral,3.7,3.1


In [144]:
new_columns = df_ann.iloc[0]
df_ann.columns = new_columns

current_columns = df_ann.columns.tolist()
old_name = current_columns[2]
new_name = 'Segment ID'

df_ann.rename(columns={old_name: new_name}, inplace=True)

df_ann.drop(index=0, inplace=True)
df_ann.reset_index(drop=True, inplace=True)

In [145]:
df_ann.head()

,start,end,Segment ID,Emotion,Valence,Arousal
0,3.5783901,11.637391,Sess01_script01_User002M_001,neutral,3.4,2.9
1,11.637391,23.334393,Sess01_script01_User002M_002,neutral,3.1,2.9
2,23.334393,31.558392,Sess01_script01_User002M_003,neutral,3.1,3
3,31.558392,42.97039,Sess01_script01_User002M_004,neutral,3.7,3.1
4,42.77729,44.86329,Sess01_script01_User001F_001,neutral,3.8,2.8


### start / end 컬럼 소숫점 셋째 자리에서 반올림

In [146]:
df_ann['start'] = df_ann['start'].astype(float).round(3)
df_ann['end'] = df_ann['end'].astype(float).round(3)

df_ann.head()

,start,end,Segment ID,Emotion,Valence,Arousal
0,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9
1,11.637,23.334,Sess01_script01_User002M_002,neutral,3.1,2.9
2,23.334,31.558,Sess01_script01_User002M_003,neutral,3.1,3
3,31.558,42.970,Sess01_script01_User002M_004,neutral,3.7,3.1
4,42.777,44.863,Sess01_script01_User001F_001,neutral,3.8,2.8


Segment ID 분리 -> 굳이 없어도 될거같기도

In [147]:
"""
# 1. 'Segment ID' 컬럼 분리
split_data = df_ann['Segment ID'].str.split('_', expand=True)

# 2. 'User' 관련 컬럼 할당 및 추출
df_ann['User_Full'] = split_data[2] # User002M, User003F 등의 값
df_ann['User'] = df_ann['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당

# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)
df_ann['Sess_Full'] = split_data[0]
df_ann['Script_Full'] = split_data[1]
df_ann['No_Full'] = split_data[3]

df_ann['Sess'] = df_ann['Sess_Full'].str.extract('(\d+)')
df_ann['Script'] = df_ann['Script_Full'].str.extract('(\d+)')
df_ann['No'] = df_ann['No_Full']

# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.
df_ann.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)

df_ann['User'] = df_ann['User'].str[-4:]
df_ann.head()
"""

"\n# 1. 'Segment ID' 컬럼 분리\nsplit_data = df_ann['Segment ID'].str.split('_', expand=True)\n\n# 2. 'User' 관련 컬럼 할당 및 추출\ndf_ann['User_Full'] = split_data[2] # User002M, User003F 등의 값\ndf_ann['User'] = df_ann['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당\n\n# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)\ndf_ann['Sess_Full'] = split_data[0]\ndf_ann['Script_Full'] = split_data[1]\ndf_ann['No_Full'] = split_data[3]\n\ndf_ann['Sess'] = df_ann['Sess_Full'].str.extract('(\\d+)')\ndf_ann['Script'] = df_ann['Script_Full'].str.extract('(\\d+)')\ndf_ann['No'] = df_ann['No_Full']\n\n# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.\ndf_ann.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)\n\ndf_ann['User'] = df_ann['User'].str[-4:]\ndf_ann.head()\n"

# 2. EDA

In [148]:
# --- 설정 변수 ---
base_dir = 'C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/EDA/Session01'
file_pattern = 'Sess01_script0{}_User001F.csv'

# 파일을 담을 빈 리스트
eda_dataframes = []

# --- 반복문 실행: script01부터 script06까지 ---
for i in range(1, 7):
    filename = file_pattern.format(i)
    file_path = os.path.join(base_dir, filename)
    
    try:
        # CSV 파일 로드 (헤더 없이, 컬럼 이름 지정)
        df_eda_F = pd.read_csv(
            file_path,
            header=None,
            # 이전에 논의된 EDA, Time, Segment ID 순서를 따릅니다.
            names=["EDA", "Time", "Segment ID"]
        )
        
        # 리스트에 추가 (아직 행 제거/타입 변환 같은 전처리는 하지 않습니다)
        eda_dataframes.append(df_eda_F)
        print(f"✅ 파일 로드 성공: {filename}")
        
    except FileNotFoundError:
        print(f"❌ 파일 찾을 수 없음: {filename}")
    except Exception as e:
        print(f"⚠️ {filename} 로드 중 오류 발생: {e}")

# --- 행 방향 최종 병합 ---
# 모든 데이터프레임을 행 방향(위아래)으로 하나의 데이터프레임으로 합칩니다.
df_eda_F = pd.concat(eda_dataframes, ignore_index=True)

✅ 파일 로드 성공: Sess01_script01_User001F.csv
✅ 파일 로드 성공: Sess01_script02_User001F.csv
✅ 파일 로드 성공: Sess01_script03_User001F.csv
✅ 파일 로드 성공: Sess01_script04_User001F.csv
✅ 파일 로드 성공: Sess01_script05_User001F.csv
✅ 파일 로드 성공: Sess01_script06_User001F.csv


In [149]:
# --- 설정 변수 ---
base_dir = 'C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/EDA/Session01'
file_pattern = 'Sess01_script0{}_User002M.csv'

# 파일을 담을 빈 리스트
eda_dataframes = []

# --- 반복문 실행: script01부터 script06까지 ---
for i in range(1, 7):
    filename = file_pattern.format(i)
    file_path = os.path.join(base_dir, filename)
    
    try:
        # CSV 파일 로드 (헤더 없이, 컬럼 이름 지정)
        df_eda_M = pd.read_csv(
            file_path,
            header=None,
            # 이전에 논의된 EDA, Time, Segment ID 순서를 따릅니다.
            names=["EDA", "Time", "Segment ID"]
        )
        
        # 리스트에 추가 (아직 행 제거/타입 변환 같은 전처리는 하지 않습니다)
        eda_dataframes.append(df_eda_M)
        print(f"✅ 파일 로드 성공: {filename}")
        
    except FileNotFoundError:
        print(f"❌ 파일 찾을 수 없음: {filename}")
    except Exception as e:
        print(f"⚠️ {filename} 로드 중 오류 발생: {e}")

# --- 행 방향 최종 병합 ---
# 모든 데이터프레임을 행 방향(위아래)으로 하나의 데이터프레임으로 합칩니다.
df_eda_M = pd.concat(eda_dataframes, ignore_index=True)

✅ 파일 로드 성공: Sess01_script01_User002M.csv
✅ 파일 로드 성공: Sess01_script02_User002M.csv
✅ 파일 로드 성공: Sess01_script03_User002M.csv
✅ 파일 로드 성공: Sess01_script04_User002M.csv
✅ 파일 로드 성공: Sess01_script05_User002M.csv
✅ 파일 로드 성공: Sess01_script06_User002M.csv


In [150]:
df_eda_F.drop(index=[0,1], inplace=True)
df_eda_F.reset_index(drop=True, inplace=True)

In [151]:
df_eda_M.drop(index=[0,1], inplace=True)
df_eda_M.reset_index(drop=True, inplace=True)

### ID 컬럼이 결측치인 row 제거

In [152]:
df_eda_M.dropna(subset=['Segment ID'], inplace=True)
df_eda_F.dropna(subset=['Segment ID'], inplace=True)

df_eda_M.reset_index(drop=True, inplace=True)
df_eda_F.reset_index(drop=True, inplace=True)

In [153]:
df_eda_M

,EDA,Time,Segment ID
0,2.856493,2020-0821-1124-17-250,Sess01_script01_User002M_001
1,2.788578,2020-0821-1124-17-500,Sess01_script01_User002M_001
2,2.678377,2020-0821-1124-17-750,Sess01_script01_User002M_001
3,2.652749,2020-0821-1124-18-000,Sess01_script01_User002M_001
4,2.645061,2020-0821-1124-18-250,Sess01_script01_User002M_001
...,...,...,...
4916,3.250973,2020-0821-1326-21-750,Sess01_script06_User002M_043
4917,3.240721,2020-0821-1326-22-000,Sess01_script06_User002M_043
4918,3.236876,2020-0821-1326-22-250,Sess01_script06_User002M_043
4919,3.225343,2020-0821-1326-22-500,Sess01_script06_User002M_043


In [154]:
df_eda_F

,EDA,Time,Segment ID
0,4.412055,2020-0821-1128-57-250,Sess01_script01_User001F_001
1,4.536246,2020-0821-1128-57-500,Sess01_script01_User001F_001
2,4.630991,2020-0821-1128-57-750,Sess01_script01_User001F_001
3,4.669401,2020-0821-1128-58-000,Sess01_script01_User001F_001
4,4.661719,2020-0821-1128-58-250,Sess01_script01_User001F_001
...,...,...,...
2949,7.971015,2020-0821-1330-32-250,Sess01_script06_User001F_017
2950,7.862194,2020-0821-1330-32-500,Sess01_script06_User001F_017
2951,7.781053,2020-0821-1330-32-750,Sess01_script06_User001F_017
2952,7.805377,2020-0821-1330-33-000,Sess01_script06_User001F_017


# 3. IBI

In [155]:
df_ibi_F = pd.read_csv('C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/IBI/Session01/Sess01_script01_User001F.csv',
                    header=None, names=["시간", "IBI", "Time", "Segment ID"])

df_ibi_M = pd.read_csv('C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/IBI/Session01/Sess01_script01_User002M.csv',
                    header=None, names=["시간", "IBI", "Time", "Segment ID"])

맨 왼쪽 컬럼이 시작 이후 시간인거같은데, 어차피 df_ann의 시간이랑 맞지 않아서 날림

In [156]:
df_ibi_F.drop(df_ibi_F.columns[0], axis=1, inplace=True)
df_ibi_M.drop(df_ibi_M.columns[0], axis=1, inplace=True)

df_ibi_M.drop(df_ibi_M.index[0], inplace=True)
df_ibi_F.drop(df_ibi_F.index[0], inplace=True)

In [157]:
df_ibi_M.dropna(subset=['Segment ID'], inplace=True)
df_ibi_F.dropna(subset=['Segment ID'], inplace=True)

df_ibi_M.reset_index(drop=True, inplace=True)
df_ibi_F.reset_index(drop=True, inplace=True)

In [158]:
df_ibi_F.head()

,IBI,Time,Segment ID
0,0.640625,2020-0821-1128-57-859,Sess01_script01_User001F_001
1,0.640625,2020-0821-1128-58-500,Sess01_script01_User001F_001
2,0.578125,2020-0821-1128-59-078,Sess01_script01_User001F_001
3,0.609375,2020-0821-1129-46-453,Sess01_script01_User001F_003
4,0.687500,2020-0821-1129-51-343,Sess01_script01_User001F_003


# 4. TEMP

In [159]:
# --- 설정 변수 ---
base_dir = 'C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/TEMP/Session01'
file_pattern = 'Sess01_script0{}_User001F.csv'

# 파일을 담을 빈 리스트
eda_dataframes = []

# --- 반복문 실행: script01부터 script06까지 ---
for i in range(1, 7):
    filename = file_pattern.format(i)
    file_path = os.path.join(base_dir, filename)
    
    try:
        # CSV 파일 로드 (헤더 없이, 컬럼 이름 지정)
        df_temp_F = pd.read_csv(
            file_path,
            header=None,
            # 이전에 논의된 EDA, Time, Segment ID 순서를 따릅니다.
            names=["TEMP", "Time", "Segment ID"]
        )
        
        # 리스트에 추가 (아직 행 제거/타입 변환 같은 전처리는 하지 않습니다)
        eda_dataframes.append(df_temp_F)
        print(f"✅ 파일 로드 성공: {filename}")
        
    except FileNotFoundError:
        print(f"❌ 파일 찾을 수 없음: {filename}")
    except Exception as e:
        print(f"⚠️ {filename} 로드 중 오류 발생: {e}")

# --- 행 방향 최종 병합 ---
# 모든 데이터프레임을 행 방향(위아래)으로 하나의 데이터프레임으로 합칩니다.
df_temp_F = pd.concat(eda_dataframes, ignore_index=True)

✅ 파일 로드 성공: Sess01_script01_User001F.csv
✅ 파일 로드 성공: Sess01_script02_User001F.csv
✅ 파일 로드 성공: Sess01_script03_User001F.csv
✅ 파일 로드 성공: Sess01_script04_User001F.csv
✅ 파일 로드 성공: Sess01_script05_User001F.csv
✅ 파일 로드 성공: Sess01_script06_User001F.csv


In [160]:
# --- 설정 변수 ---
base_dir = 'C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/KEMDy20_v1_2/TEMP/Session01'
file_pattern = 'Sess01_script0{}_User002M.csv'

# 파일을 담을 빈 리스트
eda_dataframes = []

# --- 반복문 실행: script01부터 script06까지 ---
for i in range(1, 7):
    filename = file_pattern.format(i)
    file_path = os.path.join(base_dir, filename)
    
    try:
        # CSV 파일 로드 (헤더 없이, 컬럼 이름 지정)
        df_temp_M = pd.read_csv(
            file_path,
            header=None,
            # 이전에 논의된 EDA, Time, Segment ID 순서를 따릅니다.
            names=["TEMP", "Time", "Segment ID"]
        )
        
        # 리스트에 추가 (아직 행 제거/타입 변환 같은 전처리는 하지 않습니다)
        eda_dataframes.append(df_temp_M)
        print(f"✅ 파일 로드 성공: {filename}")
        
    except FileNotFoundError:
        print(f"❌ 파일 찾을 수 없음: {filename}")
    except Exception as e:
        print(f"⚠️ {filename} 로드 중 오류 발생: {e}")

# --- 행 방향 최종 병합 ---
# 모든 데이터프레임을 행 방향(위아래)으로 하나의 데이터프레임으로 합칩니다.
df_temp_M = pd.concat(eda_dataframes, ignore_index=True)

✅ 파일 로드 성공: Sess01_script01_User002M.csv
✅ 파일 로드 성공: Sess01_script02_User002M.csv
✅ 파일 로드 성공: Sess01_script03_User002M.csv
✅ 파일 로드 성공: Sess01_script04_User002M.csv
✅ 파일 로드 성공: Sess01_script05_User002M.csv
✅ 파일 로드 성공: Sess01_script06_User002M.csv


In [161]:
df_temp_F.drop(index=[0,1], inplace=True)
df_temp_F.reset_index(drop=True, inplace=True)

df_temp_M.drop(index=[0,1], inplace=True)
df_temp_M.reset_index(drop=True, inplace=True)

df_temp_M.dropna(subset=['Segment ID'], inplace=True)
df_temp_F.dropna(subset=['Segment ID'], inplace=True)

df_temp_M.reset_index(drop=True, inplace=True)
df_temp_F.reset_index(drop=True, inplace=True)

In [162]:
df_temp_F

,TEMP,Time,Segment ID
0,34.55,2020-0821-1128-57-250,Sess01_script01_User001F_001
1,34.55,2020-0821-1128-57-500,Sess01_script01_User001F_001
2,34.55,2020-0821-1128-57-750,Sess01_script01_User001F_001
3,34.55,2020-0821-1128-58-000,Sess01_script01_User001F_001
4,34.55,2020-0821-1128-58-250,Sess01_script01_User001F_001
...,...,...,...
2949,34.43,2020-0821-1330-32-250,Sess01_script06_User001F_017
2950,34.43,2020-0821-1330-32-500,Sess01_script06_User001F_017
2951,34.43,2020-0821-1330-32-750,Sess01_script06_User001F_017
2952,34.43,2020-0821-1330-33-000,Sess01_script06_User001F_017


In [163]:
df_temp_M

,TEMP,Time,Segment ID
0,34.81,2020-0821-1124-17-250,Sess01_script01_User002M_001
1,34.81,2020-0821-1124-17-500,Sess01_script01_User002M_001
2,34.81,2020-0821-1124-17-750,Sess01_script01_User002M_001
3,34.79,2020-0821-1124-18-000,Sess01_script01_User002M_001
4,34.79,2020-0821-1124-18-250,Sess01_script01_User002M_001
...,...,...,...
4916,36.07,2020-0821-1326-21-750,Sess01_script06_User002M_043
4917,36.05,2020-0821-1326-22-000,Sess01_script06_User002M_043
4918,36.05,2020-0821-1326-22-250,Sess01_script06_User002M_043
4919,36.05,2020-0821-1326-22-500,Sess01_script06_User002M_043


IBI 데이터가 있는 발화가 있고, 누락된거같은 발화도 있음.\
이 부분을 어떻게 처리할 것인가?

일단 IBI는 냅두고, EDA, TEMP만 합치자

In [164]:
df_et_M = pd.merge(df_eda_M, df_temp_M, how='inner', on=['Segment ID', 'Time'])
df_et_F = pd.merge(df_eda_F, df_temp_F, how='inner', on=['Segment ID', 'Time'])

In [165]:
order = ["Segment ID", "Time", "EDA", "TEMP"]

df_et_M = df_et_M[order]
df_et_F = df_et_F[order]

In [166]:
df_et_F.head()

,Segment ID,Time,EDA,TEMP
0,Sess01_script01_User001F_001,2020-0821-1128-57-250,4.412055,34.55
1,Sess01_script01_User001F_001,2020-0821-1128-57-500,4.536246,34.55
2,Sess01_script01_User001F_001,2020-0821-1128-57-750,4.630991,34.55
3,Sess01_script01_User001F_001,2020-0821-1128-58-000,4.669401,34.55
4,Sess01_script01_User001F_001,2020-0821-1128-58-250,4.661719,34.55


In [167]:
df_et_M.head()

,Segment ID,Time,EDA,TEMP
0,Sess01_script01_User002M_001,2020-0821-1124-17-250,2.856493,34.81
1,Sess01_script01_User002M_001,2020-0821-1124-17-500,2.788578,34.81
2,Sess01_script01_User002M_001,2020-0821-1124-17-750,2.678377,34.81
3,Sess01_script01_User002M_001,2020-0821-1124-18-000,2.652749,34.79
4,Sess01_script01_User002M_001,2020-0821-1124-18-250,2.645061,34.79


df_et도 Segment ID 분리

In [168]:
"""
# 1. 'Segment ID' 컬럼 분리
split_data = df_et_F['Segment ID'].str.split('_', expand=True)

# 2. 'User' 관련 컬럼 할당 및 추출
df_et_F['User_Full'] = split_data[2] # User002M, User003F 등의 값
df_et_F['User'] = df_et_F['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당

# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)
df_et_F['Sess_Full'] = split_data[0]
df_et_F['Script_Full'] = split_data[1]
df_et_F['No_Full'] = split_data[3]

df_et_F['Sess'] = df_et_F['Sess_Full'].str.extract('(\d+)')
df_et_F['Script'] = df_et_F['Script_Full'].str.extract('(\d+)')
df_et_F['No'] = df_et_F['No_Full']

# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.
df_et_F.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)

df_et_F['User'] = df_et_F['User'].str[-4:]
df_et_F.head()
"""

"\n# 1. 'Segment ID' 컬럼 분리\nsplit_data = df_et_F['Segment ID'].str.split('_', expand=True)\n\n# 2. 'User' 관련 컬럼 할당 및 추출\ndf_et_F['User_Full'] = split_data[2] # User002M, User003F 등의 값\ndf_et_F['User'] = df_et_F['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당\n\n# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)\ndf_et_F['Sess_Full'] = split_data[0]\ndf_et_F['Script_Full'] = split_data[1]\ndf_et_F['No_Full'] = split_data[3]\n\ndf_et_F['Sess'] = df_et_F['Sess_Full'].str.extract('(\\d+)')\ndf_et_F['Script'] = df_et_F['Script_Full'].str.extract('(\\d+)')\ndf_et_F['No'] = df_et_F['No_Full']\n\n# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.\ndf_et_F.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)\n\ndf_et_F['User'] = df_et_F['User'].str[-4:]\ndf_et_F.head()\n"

In [169]:
"""
# 1. 'Segment ID' 컬럼 분리
split_data = df_et_M['Segment ID'].str.split('_', expand=True)

# 2. 'User' 관련 컬럼 할당 및 추출
df_et_M['User_Full'] = split_data[2] # User002M, User003F 등의 값
df_et_M['User'] = df_et_M['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당

# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)
df_et_M['Sess_Full'] = split_data[0]
df_et_M['Script_Full'] = split_data[1]
df_et_M['No_Full'] = split_data[3]

df_et_M['Sess'] = df_et_M['Sess_Full'].str.extract('(\d+)')
df_et_M['Script'] = df_et_M['Script_Full'].str.extract('(\d+)')
df_et_M['No'] = df_et_M['No_Full']

# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.
df_et_M.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)

df_et_M['User'] = df_et_M['User'].str[-4:]
df_et_M.head()
"""

"\n# 1. 'Segment ID' 컬럼 분리\nsplit_data = df_et_M['Segment ID'].str.split('_', expand=True)\n\n# 2. 'User' 관련 컬럼 할당 및 추출\ndf_et_M['User_Full'] = split_data[2] # User002M, User003F 등의 값\ndf_et_M['User'] = df_et_M['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당\n\n# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)\ndf_et_M['Sess_Full'] = split_data[0]\ndf_et_M['Script_Full'] = split_data[1]\ndf_et_M['No_Full'] = split_data[3]\n\ndf_et_M['Sess'] = df_et_M['Sess_Full'].str.extract('(\\d+)')\ndf_et_M['Script'] = df_et_M['Script_Full'].str.extract('(\\d+)')\ndf_et_M['No'] = df_et_M['No_Full']\n\n# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.\ndf_et_M.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)\n\ndf_et_M['User'] = df_et_M['User'].str[-4:]\ndf_et_M.head()\n"

df_et_M, df_et_F 병합(row 방향으로)

In [170]:
df_et = pd.concat([df_et_M, df_et_F], axis=0, ignore_index=True)
df_et

,Segment ID,Time,EDA,TEMP
0,Sess01_script01_User002M_001,2020-0821-1124-17-250,2.856493,34.81
1,Sess01_script01_User002M_001,2020-0821-1124-17-500,2.788578,34.81
2,Sess01_script01_User002M_001,2020-0821-1124-17-750,2.678377,34.81
3,Sess01_script01_User002M_001,2020-0821-1124-18-000,2.652749,34.79
4,Sess01_script01_User002M_001,2020-0821-1124-18-250,2.645061,34.79
...,...,...,...,...
7870,Sess01_script06_User001F_017,2020-0821-1330-32-250,7.971015,34.43
7871,Sess01_script06_User001F_017,2020-0821-1330-32-500,7.862194,34.43
7872,Sess01_script06_User001F_017,2020-0821-1330-32-750,7.781053,34.43
7873,Sess01_script06_User001F_017,2020-0821-1330-33-000,7.805377,34.43


In [171]:
df = pd.merge(df_ann, df_et, on="Segment ID", how='right')
df.head()

,start,end,Segment ID,Emotion,Valence,Arousal,Time,EDA,TEMP
0,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-250,2.856493,34.81
1,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-500,2.788578,34.81
2,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-750,2.678377,34.81
3,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-000,2.652749,34.79
4,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-250,2.645061,34.79


In [172]:
# 1. 'Segment ID' 컬럼 분리
split_data = df['Segment ID'].str.split('_', expand=True)

# 2. 'User' 관련 컬럼 할당 및 추출
df['User_Full'] = split_data[2] # User002M, User003F 등의 값
df['User'] = df['User_Full'] # 'User' 컬럼에 User002M 형태의 값을 그대로 할당

# 3. 'Sess', 'Script', 'No' 컬럼도 다시 생성 (전체 로직의 일관성을 위해)
df['Sess_Full'] = split_data[0]
df['Script_Full'] = split_data[1]
df['No_Full'] = split_data[3]

df['Script'] = df['Script_Full'].str.extract('(\d+)')

# 4. 임시로 만든 'Full' 컬럼을 삭제합니다.
df.drop(columns=['Sess_Full', 'Script_Full', 'User_Full', 'No_Full'], inplace=True)

df.head()

,start,end,Segment ID,Emotion,Valence,Arousal,Time,EDA,TEMP,User,Script
0,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-250,2.856493,34.81,User002M,01
1,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-500,2.788578,34.81,User002M,01
2,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-750,2.678377,34.81,User002M,01
3,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-000,2.652749,34.79,User002M,01
4,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-250,2.645061,34.79,User002M,01


In [173]:
df.sort_values(
    by=['Script','start', 'Time'],
    ascending=[True, True, True],
    inplace=True
)
df

,start,end,Segment ID,Emotion,Valence,Arousal,Time,EDA,TEMP,User,Script
0,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-250,2.856493,34.81,User002M,01
1,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-500,2.788578,34.81,User002M,01
2,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-17-750,2.678377,34.81,User002M,01
3,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-000,2.652749,34.79,User002M,01
4,3.578,11.637,Sess01_script01_User002M_001,neutral,3.4,2.9,2020-0821-1124-18-250,2.645061,34.79,User002M,01
...,...,...,...,...,...,...,...,...,...,...,...
7870,338.866,347.585,Sess01_script06_User001F_017,neutral,3,3.4,2020-0821-1330-32-250,7.971015,34.43,User001F,06
7871,338.866,347.585,Sess01_script06_User001F_017,neutral,3,3.4,2020-0821-1330-32-500,7.862194,34.43,User001F,06
7872,338.866,347.585,Sess01_script06_User001F_017,neutral,3,3.4,2020-0821-1330-32-750,7.781053,34.43,User001F,06
7873,338.866,347.585,Sess01_script06_User001F_017,neutral,3,3.4,2020-0821-1330-33-000,7.805377,34.43,User001F,06


In [174]:
df.drop(columns=['start', 'end', 'Script'], axis=1, inplace=True)

In [175]:
order = ["Segment ID", "Time", "EDA", "TEMP", "Emotion", "Valence", "Arousal"]

df = df[order]

In [176]:
df

,Segment ID,Time,EDA,TEMP,Emotion,Valence,Arousal
0,Sess01_script01_User002M_001,2020-0821-1124-17-250,2.856493,34.81,neutral,3.4,2.9
1,Sess01_script01_User002M_001,2020-0821-1124-17-500,2.788578,34.81,neutral,3.4,2.9
2,Sess01_script01_User002M_001,2020-0821-1124-17-750,2.678377,34.81,neutral,3.4,2.9
3,Sess01_script01_User002M_001,2020-0821-1124-18-000,2.652749,34.79,neutral,3.4,2.9
4,Sess01_script01_User002M_001,2020-0821-1124-18-250,2.645061,34.79,neutral,3.4,2.9
...,...,...,...,...,...,...,...
7870,Sess01_script06_User001F_017,2020-0821-1330-32-250,7.971015,34.43,neutral,3,3.4
7871,Sess01_script06_User001F_017,2020-0821-1330-32-500,7.862194,34.43,neutral,3,3.4
7872,Sess01_script06_User001F_017,2020-0821-1330-32-750,7.781053,34.43,neutral,3,3.4
7873,Sess01_script06_User001F_017,2020-0821-1330-33-000,7.805377,34.43,neutral,3,3.4


Time 컬럼 0.25 0.5 0.75 ... 로 바꿈

In [177]:
# 1. 데이터프레임의 총 행 수를 계산합니다.
num_rows = len(df)

# 2. 0.25, 0.50, 0.75, 1.00 ... 와 같이 0.25씩 증가하는 시퀀스를 생성합니다.
# np.arange(1, num_rows + 1)은 [1, 2, 3, ...] 정수 배열을 생성합니다.
new_times = np.arange(1, num_rows + 1) * 0.25

# 3. Time 컬럼을 새로운 값으로 덮어씁니다.
df['Time'] = new_times

df

C:\Users\cloud\AppData\Local\Temp\ipykernel_19972\2110395832.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Time'] = new_times


,Segment ID,Time,EDA,TEMP,Emotion,Valence,Arousal
0,Sess01_script01_User002M_001,0.25,2.856493,34.81,neutral,3.4,2.9
1,Sess01_script01_User002M_001,0.50,2.788578,34.81,neutral,3.4,2.9
2,Sess01_script01_User002M_001,0.75,2.678377,34.81,neutral,3.4,2.9
3,Sess01_script01_User002M_001,1.00,2.652749,34.79,neutral,3.4,2.9
4,Sess01_script01_User002M_001,1.25,2.645061,34.79,neutral,3.4,2.9
...,...,...,...,...,...,...,...
7870,Sess01_script06_User001F_017,1967.75,7.971015,34.43,neutral,3,3.4
7871,Sess01_script06_User001F_017,1968.00,7.862194,34.43,neutral,3,3.4
7872,Sess01_script06_User001F_017,1968.25,7.781053,34.43,neutral,3,3.4
7873,Sess01_script06_User001F_017,1968.50,7.805377,34.43,neutral,3,3.4


In [178]:
df.rename(
    columns={'Segment ID': 'Segment_ID'}, 
    inplace=True
)

df

C:\Users\cloud\AppData\Local\Temp\ipykernel_19972\2811175449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(


,Segment_ID,Time,EDA,TEMP,Emotion,Valence,Arousal
0,Sess01_script01_User002M_001,0.25,2.856493,34.81,neutral,3.4,2.9
1,Sess01_script01_User002M_001,0.50,2.788578,34.81,neutral,3.4,2.9
2,Sess01_script01_User002M_001,0.75,2.678377,34.81,neutral,3.4,2.9
3,Sess01_script01_User002M_001,1.00,2.652749,34.79,neutral,3.4,2.9
4,Sess01_script01_User002M_001,1.25,2.645061,34.79,neutral,3.4,2.9
...,...,...,...,...,...,...,...
7870,Sess01_script06_User001F_017,1967.75,7.971015,34.43,neutral,3,3.4
7871,Sess01_script06_User001F_017,1968.00,7.862194,34.43,neutral,3,3.4
7872,Sess01_script06_User001F_017,1968.25,7.781053,34.43,neutral,3,3.4
7873,Sess01_script06_User001F_017,1968.50,7.805377,34.43,neutral,3,3.4


감정 컬럼에 ; 들어간거 지우기

In [180]:
df = df[~df['Emotion'].str.contains(';', na=False)]

In [181]:
df["Emotion"].unique()

array(['neutral', 'happy'], dtype=object)

## csv로 저장

In [182]:
import os

output_dir = 'C:/Users/cloud/Desktop/2025/응용통계학과 분석 공모전/전처리/데이터'
output_filename = 'Sess01.csv'
output_path = os.path.join(output_dir, output_filename)

df.to_csv(output_path, index=False, encoding='utf-8')